<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,  # Specify radius in meters
    color='#3388ff',  # Blue color in hex
    fill=True,
    fill_color='#3388ff',  # Blue fill color
    fill_opacity=0.4,  # Opacity of the fill color
).add_child(folium.Popup('NASA Johnson Space Center'))

# Add the circle to the map
site_map.add_child(circle)

# Create an icon marker at NASA Johnson Space Center with a custom icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),  # Size of the icon
        icon_anchor=(0, 0),  # Anchor position of the icon
        html='<div style="font-size: 12; color:#3388ff;"><b>%s</b></div>' % 'NASA JSC',  # HTML content for the icon
    )
)

# Add the marker to the map
site_map.add_child(marker)

# Display the map
site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [19]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Select relevant columns and group by launch site to get the first occurrence

# Fetch the SpaceX launch site data from the provided URL
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())

# Read the CSV data into a DataFrame
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant columns and group by launch site to get the first occurrence
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Coordinates for NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a Folium map centered at NASA Johnson Space Center's coordinates
site_map = folium.Map(location=nasa_coordinate, zoom_start=5, zoom_control=True)

# Add blue circle for NASA Johnson Space Center
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,  # Specify radius in meters
    color='#3388ff',  # Blue color in hex
    fill=True,
    fill_color='#3388ff',  # Blue fill color
    fill_opacity=0.4,  # Opacity of the fill color
).add_child(folium.Popup('NASA Johnson Space Center'))  # Add popup label

site_map.add_child(circle)

# Loop through each launch site in the launch_sites_df DataFrame
for index, site in launch_sites_df.iterrows():
    # Extract the coordinates and name of the launch site
    site_coordinate = [site['Lat'], site['Long']]
    site_name = site['Launch Site']
    
    # Create a Circle object for the launch site with a popup label
    circle = folium.Circle(
        site_coordinate,
        radius=1000,  # Specify radius in meters
        color='#3388ff',  # Blue color in hex
        fill=True,
        fill_color='#3388ff',  # Blue fill color
        fill_opacity=0.4,  # Opacity of the fill color
    ).add_child(folium.Popup(site_name))  # Add popup label with site name
    
    # Add the circle to the map
    site_map.add_child(circle)
    
    # Create a Marker with a custom icon for the launch site
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),  # Size of the icon
            icon_anchor=(0, 0),  # Anchor position of the icon
            html='<div style="font-size: 12; color:#3388ff;"><b>%s</b></div>' % site_name,  # HTML content for the icon
        )
    )
    
    # Add the marker to the map
    site_map.add_child(marker)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [59]:
# Task 2: Mark the success/failed launches for each site on the map
# Calculate success rates for each launch site
site_success = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
site_success.rename(columns={'class': 'Success Rate'}, inplace=True)

# Merge success rates with launch sites DataFrame
launch_sites_df = pd.merge(launch_sites_df, site_success, on='Launch Site')

# Loop through each launch site in the launch_sites_df DataFrame
for index, site in launch_sites_df.iterrows():
    # Extract the success rate of the launch site
    success_rate = site['Success Rate']
    
    # Determine the color of the marker based on success rate
    if success_rate >= 0.8:
        marker_color = 'green'
    elif success_rate >= 0.6:
        marker_color = 'orange'
    else:
        marker_color = 'red'
    
    # Create a Circle object for the launch site with a popup label
    circle = folium.Circle(
        [site['Lat'], site['Long']],
        radius=1000,  # Specify radius in meters
        color=marker_color,  # Color based on success rate
        fill=True,
        fill_color=marker_color,  # Fill color based on success rate
        fill_opacity=0.6,  # Opacity of the fill color
    ).add_child(folium.Popup(f'{site["Launch Site"]}<br>Success Rate: {success_rate:.2f}'))  # Add popup label with site name and success rate
    
    # Add the circle to the map
    site_map.add_child(circle)
    
    # Create a Marker with a custom icon for the launch site
    marker = folium.map.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20, 20),  # Size of the icon
            icon_anchor=(0, 0),  # Anchor position of the icon
            html=f'<div style="font-size: 12; color:{marker_color};"><b>{site["Launch Site"]}</b></div>',  # HTML content for the icon
        )
    )
    
    # Add the marker to the map
    site_map.add_child(marker)

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [23]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [60]:
# Loop through each launch site in launch_sites_df
for index, site in launch_sites_df.iterrows():
    # Filter records for the current launch site
    site_records = spacex_df[spacex_df['Launch Site'] == site['Launch Site']]
    
    # Calculate success rate for the launch site
    success_rate = site_records['class'].mean()
    
    # Determine the marker color based on success rate
    if success_rate >= 0.8:
        marker_color = 'green'
    elif success_rate >= 0.6:
        marker_color = 'orange'
    else:
        marker_color = 'red'
    
    # Create a Circle object for the launch site with a popup label
    circle = folium.Circle(
        [site['Lat'], site['Long']],
        radius=1000,  # Specify radius in meters
        color=marker_color,  # Color based on success rate
        fill=True,
        fill_color=marker_color,  # Fill color based on success rate
        fill_opacity=0.6,  # Opacity of the fill color
    ).add_child(folium.Popup(f'{site["Launch Site"]}<br>Success Rate: {success_rate:.2f}'))  # Add popup label with site name and success rate
    
    # Add the circle to the map
    site_map.add_child(circle)
    
    # Create markers for successful and failed launches
    for _, record in site_records.iterrows():
        if record['class'] == 1:
            marker_color = 'green'  # Successful launch
        else:
            marker_color = 'red'  # Failed launch
        
       # Create a Marker for each launch record
        marker = folium.map.Marker(
            [record['Lat'], record['Long']],
            icon=folium.Icon(color=marker_color)
        )
        
        # Add the marker to the map
        site_map.add_child(marker)

# Display the map
site_map


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [61]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Group by Launch Site and calculate the success rate
site_success = spacex_df.groupby('Launch Site')['class'].mean().reset_index()

# Merge success rates with launch sites DataFrame
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Merge success rate back into launch sites DataFrame
launch_sites_df = pd.merge(launch_sites_df, site_success, on='Launch Site')

# Define a function to assign marker color based on class value
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
launch_sites_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [62]:
import folium
import folium
from folium.plugins import MarkerCluster

# Coordinates for NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a Folium map centered at NASA Johnson Space Center's coordinates
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add smaller red circles for each launch site
for index, site in launch_sites_df.iterrows():
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=50,  # Reduced radius to make circles smaller
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.4,  # Adjusted opacity
        popup=site['Launch Site']
    ).add_to(site_map)

# Loop through each row in spacex_df for success and failure markers
for index, record in spacex_df.iterrows():
    if record['class'] == 1:
        icon_color = 'green'  # Success
    else:
        icon_color = 'red'  # Failure
    
    # Create a Marker with its coordinates and custom icon
    icon = folium.Icon(color='white', icon_color=icon_color)
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=icon,
        popup=f"Outcome: {'Successful' if record['class'] == 1 else 'Failed'}<br>Launch Site: {record['Launch Site']}"
    )
    
    # Add the Marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Display the map
site_map

   





Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [63]:
# TASK 3: Calculate the distances between a launch site to its proximities
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of Earth in kilometers
    
    return distance

# Example usage:
launch_site_lat = launch_sites_df.loc[0, 'Lat']  # Latitude of the launch site (for example, the first site)
launch_site_lon = launch_sites_df.loc[0, 'Long']  # Longitude of the launch site

# Calculate distances between the launch site and all other sites in launch_sites_df
distances = []
for index, site in launch_sites_df.iterrows():
    distance = calculate_distance(launch_site_lat, launch_site_lon, site['Lat'], site['Long'])
    distances.append(distance)

# Add distances as a new column to launch_sites_df
launch_sites_df['Distance'] = distances

# Display launch_sites_df with distances calculated
launch_sites_df



,Launch Site,Lat,Long,class,marker_color,Distance
0,CCAFS LC-40,28.562302,-80.577356,0.269231,red,0.000000
1,CCAFS SLC-40,28.563197,-80.576820,0.428571,red,0.112488
2,KSC LC-39A,28.573255,-80.646895,0.769231,red,6.899304
3,VAFB SLC-4E,34.632834,-120.610745,0.400000,red,3825.840279


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [64]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [65]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [44]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56283  Lon: -80.56779
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [71]:
import folium
from folium.plugins import MarkerCluster
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates for NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Coordinates for Florida launch site
florida_launch_coordinate = [28.56317, -80.57684]

# Coordinate of the coastline
coastline_coordinate = [28.56283, -80.56779]

# Create a Folium map centered at NASA Johnson Space Center's coordinates
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add smaller red circles for each launch site
for index, site in launch_sites_df.iterrows():
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=50,  # Reduced radius to make circles smaller
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.4,  # Adjusted opacity
        popup=site['Launch Site']
    ).add_to(site_map)

# Loop through each row in spacex_df for success and failure markers
for index, record in spacex_df.iterrows():
    if record['class'] == 1:
        icon_color = 'green'  # Success
    else:
        icon_color = 'red'  # Failure
    
    # Create a Marker with its coordinates and custom icon
    icon = folium.Icon(color='white', icon_color=icon_color)
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=icon,
        popup=f"Outcome: {'Successful' if record['class'] == 1 else 'Failed'}<br>Launch Site: {record['Launch Site']}"
    )
    
    # Add the Marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add blue line from launch site to coastline
folium.PolyLine(
    locations=[florida_launch_coordinate, coastline_coordinate],
    color='blue',
    weight=2
).add_to(site_map)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [77]:
import folium
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates for the Florida launch site
florida_launch_coordinate = [28.56317, -80.57684]

# Coordinates for the closest coastline point
coastline_coordinate = [28.56283, -80.56779]

# Create a Folium map centered at the launch site's coordinates
site_map = folium.Map(location=florida_launch_coordinate, zoom_start=10)

# Add MousePosition plugin to display coordinates on mouse hover
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = folium.plugins.MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter
)
site_map.add_child(mouse_position)

# Add Marker for the Florida launch site
folium.Marker(
    location=florida_launch_coordinate,
    icon=folium.Icon(color='red'),
    popup="Florida Launch Site"
).add_to(site_map)

# Add Marker for the closest coastline point
folium.Marker(
    location=coastline_coordinate,
    icon=folium.Icon(color='blue'),
    popup="Closest Coastline"
).add_to(site_map)

# Add Polyline (blue line) from launch site to coastline
folium.PolyLine(
    locations=[florida_launch_coordinate, coastline_coordinate],
    color='blue',
    weight=2
).add_to(site_map)

# Calculate distance from launch site to coastline
distance_to_coastline = calculate_distance(florida_launch_coordinate[0], florida_launch_coordinate[1],
                                           coastline_coordinate[0], coastline_coordinate[1])

# Add distance information as a popup on the blue line
popup_content = f"Distance to Coastline: {distance_to_coastline:.2f} KM"
folium.Marker(
    location=[(florida_launch_coordinate[0] + coastline_coordinate[0]) / 2, 
              (florida_launch_coordinate[1] + coastline_coordinate[1]) / 2],
    icon=folium.DivIcon(html=f'<div style="font-size: 12; color:#0000FF;"><b>{popup_content}</b></div>')
).add_to(site_map)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [78]:
import folium
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates for the Florida launch site
florida_launch_coordinate = [28.56317, -80.57684]

# Coordinates for Space Center Regional Airport
airport_coordinate = [28.51074, -80.79878]

# Coordinates for US 1 and FL 405
us_1_fl_405_coordinate = [28.5257, -80.78848]

# Coordinates for Railline
railline_coordinate = [28.52478, -80.78994]

# Create a Folium map centered at the launch site's coordinates
site_map = folium.Map(location=florida_launch_coordinate, zoom_start=10)

# Add MousePosition plugin to display coordinates on mouse hover
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = folium.plugins.MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter
)
site_map.add_child(mouse_position)

# Add Marker for the Florida launch site
folium.Marker(
    location=florida_launch_coordinate,
    icon=folium.Icon(color='red'),
    popup="Florida Launch Site"
).add_to(site_map)

# Add Markers for other locations
locations = {
    "Space Center Regional Airport": airport_coordinate,
    "US 1 and FL 405": us_1_fl_405_coordinate,
    "Railline": railline_coordinate
}

for location, coords in locations.items():
    folium.Marker(
        location=coords,
        icon=folium.Icon(color='green'),
        popup=location
    ).add_to(site_map)

# Draw Polylines (lines) from launch site to each location
for location, coords in locations.items():
    folium.PolyLine(
        locations=[florida_launch_coordinate, coords],
        color='blue',
        weight=2,
        popup=f"Distance to {location}: {calculate_distance(florida_launch_coordinate[0], florida_launch_coordinate[1], coords[0], coords[1]):.2f} KM"
    ).add_to(site_map)

# Display the map
site_map



After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
